In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
from skimage import color, morphology
from numpy import linalg as LA
import sys
from skimage.morphology import skeletonize
%matplotlib inline

# following code is to segment the tissue with the specified number so that each segmented part should have the same area

In [2]:
# to add blank area around original image to give sufficient space to draw orthogonal line to the manually drawn axis
def add_blank(input_img, input_axis):
    h = int(input_img.shape[0] / 5)
    w = int(input_img.shape[1] / 2)
    
    blank_side = np.zeros((input_img.shape[0], w)).astype(np.uint8)
    blank_updown = np.zeros((h, 2*w + input_img.shape[1])).astype(np.uint8)
    
    newdst, newaxis = cv2.hconcat([blank_side, input_img]), cv2.hconcat([blank_side, input_axis])
    newdst2, newaxis2 = cv2.hconcat([newdst, blank_side]), cv2.hconcat([newaxis, blank_side])
    newdst3, newaxis3 = cv2.vconcat([blank_updown, newdst2]), cv2.vconcat([blank_updown, newaxis2])
    newdst4, newaxis4 = cv2.vconcat([newdst3, blank_updown]), cv2.vconcat([newaxis3, blank_updown])
    
    return newdst4, newaxis4
    
    

In [3]:
# each coordinate on axis are reordered according to the distance from the reference point
def order_points(axis):  #ref_point[x,y]
    img_th=np.where(axis>100,255,0).astype(np.uint8)
    skeleton=(skeletonize(img_th/255)*255).astype(np.uint8)
    pos=np.array(list(zip(*np.where(skeleton>100))))
    pos_x, pos_y= pos[:,1], pos[:,0]
    
    length=[]
    for i in range(len(pos_x)):
        l=np.sqrt((pos_x[i]-ref_point[0])**2 + (pos_y[i]-ref_point[1])**2)    #distance from ref_point
        length.append(l)
    
    pos_x_list, pos_y_list= pos_x.tolist(), pos_y.tolist() 
        
    zipped=list(zip(pos_x_list, pos_y_list, length))
    zipped.sort(key=lambda x: x[2]) #reorder the list(zipped) according to length
    zipped_array=np.array(zipped)
    pos_x_ordered, pos_y_ordered = zipped_array[:,0], zipped_array[:,1]
    
    return pos_x_ordered, pos_y_ordered
    


In [4]:
# get all the possible orthogonal lines to the anterior-posterior axis
def get_ortho_slope(pos_x_ordered, pos_y_ordered):

    posx_sub=pos_x_ordered[skip_value:len(pos_x_ordered):skip_value]
    posy_sub=pos_y_ordered[skip_value:len(pos_y_ordered):skip_value]



    slope_list=[]
    ortho_slope_list=[]
    
    for i in range(len(posx_sub)):
    
        if i*skip_value + skip_value - slope_cal <= 0:
            
            if pos_x_ordered[i*skip_value + skip_value + slope_cal] == pos_x_ordered[0]:
                slope = 1000
            else:
                slope = (pos_y_ordered[i*skip_value + skip_value + slope_cal] - pos_y_ordered[0]) / (pos_x_ordered[i*skip_value + skip_value + slope_cal] - pos_x_ordered[0])
                
               
        elif i*skip_value + skip_value + slope_cal >= len(pos_x_ordered):
            
            if pos_x_ordered[-1] == pos_x_ordered[i*skip_value + skip_value -slope_cal]:
                slope = 1000
            else:
                slope=(pos_y_ordered[-1] - pos_y_ordered[i*skip_value + skip_value - slope_cal])/(pos_x_ordered[-1] - pos_x_ordered[i*skip_value + skip_value -slope_cal])    
    
        
        elif pos_x_ordered[i*skip_value + skip_value + slope_cal] == pos_x_ordered[i*skip_value + skip_value - slope_cal]:
            slope=1000  # ortho_slope cannot be calculated. Replaced by 1000
                
        else:
            slope=(pos_y_ordered[i*skip_value + skip_value + slope_cal] - pos_y_ordered[i*skip_value + skip_value - slope_cal])/(pos_x_ordered[i*skip_value + skip_value + slope_cal] - pos_x_ordered[i*skip_value + skip_value - slope_cal])
        
        slope_list.append(slope)
    
        if slope==0:
            ortho_slope=1000  # ortho_slope cannot be calculated. Replaced by 1000
        else:
            ortho_slope=-1/slope
    
        ortho_slope_list.append(ortho_slope)
    
    posx_sub = posx_sub.astype(int)
    posy_sub = posy_sub.astype(int)
    
    return(posx_sub, posy_sub, ortho_slope_list)


    
    
        
    

    
        

In [5]:
# draw all the possibel orthogonal lines and get segmented area
def get_segmented_area (posx_sub, posy_sub, ortho_slope_list):
     
    area_list=[]
    posx_sub_updated=[]
    posy_sub_updated=[]
    ortho_slope_updated=[] 
    
    
    for i in range(len(posx_sub)):
        dst_img_th=np.where(dst_img>100,255,0).astype(np.uint8)     
        ortho_slope=ortho_slope_list[i]
        target_x, target_y = posx_sub[i], posy_sub[i]
    
        end_plus_x = int(target_x + line_length / np.sqrt(1 + ortho_slope**2))
        end_plus_y = int(target_y + line_length*ortho_slope / np.sqrt(1 + ortho_slope**2))
        
        end_minus_x = int(target_x - line_length / np.sqrt(1 + ortho_slope**2))
        end_minus_y = int(target_y - line_length*ortho_slope / np.sqrt(1 + ortho_slope**2))
        
        if i == len(posx_sub) - 1:
            continue
        else:
            end_horizon_x = posx_sub[i + 1]
            end_horizon_y = posy_sub[i + 1]
        
        horizon_vector = np.array([end_horizon_x - target_x, end_horizon_y - target_y, 0])
        end_plus_vector = np.array([end_plus_x - target_x, end_plus_y - target_y, 0])
        end_minus_vector = np.array([end_minus_x - target_x, end_minus_y - target_y, 0])  
               
        z_value = np.cross(horizon_vector, end_plus_vector)[2]
        
        if z_value <= 0:
            cv2.line(dst_img_th, (target_x, target_y), (end_plus_x, end_plus_y), (0,0,0), 5)
        else:
            cv2.line(dst_img_th, (target_x, target_y), (end_minus_x, end_minus_y), (0,0,0), 5)
        
        
                
        contours, hierarchy = cv2.findContours(dst_img_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                       
        sub_area=[]
                
        for j, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            sub_area.append(area)
                    
        if len(sub_area) != 2:
            continue
        
        cx_list=[]
        cy_list=[]
        
        for k, contour in enumerate(contours):
            m = cv2.moments(contour)
            cx = m["m10"] / m["m00"]
            cy = m["m01"] / m["m00"]
            
            cx_list.append(cx)
            cy_list.append(cy)
        
        
            
        
        p = (ref_point[0]-cx_list[0])**2 + (ref_point[1]-cy_list[0])**2
        q = (ref_point[0]-cx_list[1])**2 + (ref_point[1]-cy_list[1])**2
        
        
        if p-q>=0:
            ob_idx=1
        else:
            ob_idx=0
        
        area_list.append(sub_area[ob_idx])
        posx_sub_updated.append(posx_sub[i])
        posy_sub_updated.append(posy_sub[i])
        ortho_slope_updated.append(ortho_slope_list[i])
        
        
    return posx_sub_updated, posy_sub_updated, ortho_slope_updated, area_list

        
        
    
    
        
    

    
        

In [6]:
# select the orthogonal line to divide tissue with specified value
def segmenter_list(posx_sub_updated, posy_sub_updated, ortho_slope_updated, area_list):
    dst_img_th=np.where(dst_img>100,255,0).astype(np.uint8)
    dst_img_area=cv2.countNonZero(dst_img_th)
    
    single_area=dst_img_area/dividing_num
    
    x_final=[]
    y_final=[]
    ortho_slope_final=[]
    
    for i in range(dividing_num-1):
        ref_area = single_area + i*single_area
        idx=np.argmin(np.abs(np.array(area_list)-ref_area))
        
        x_final.append(posx_sub_updated[idx])
        y_final.append(posy_sub_updated[idx])
        ortho_slope_final.append(ortho_slope_updated[idx])
    
    return x_final, y_final, ortho_slope_final
        
        
        
    
    
        
    

    
        

In [7]:
#draw line and save the image
def draw_line(x_final, y_final, ortho_slope_final):
    dst_img_th=np.where(dst_img>100,255,0).astype(np.uint8)
    
    
    for i in range(len(x_final)):
                
        ortho_slope=ortho_slope_final[i]
        target_x, target_y = x_final[i], y_final[i]
    
        end_plus_x = int(target_x + (line_length)/np.sqrt(1+(ortho_slope)**2))
        end_plus_y = int(target_y + (line_length)*(ortho_slope)/np.sqrt(1+(ortho_slope)**2))
        
        end_minus_x = int(target_x - (line_length)/np.sqrt(1+(ortho_slope)**2))
        end_minus_y = int(target_y - (line_length)*(ortho_slope)/np.sqrt(1+(ortho_slope)**2))
        
        end_plus_vector = np.array([end_plus_x - target_x, end_plus_y - target_y, 0])
        end_minus_vector = np.array([end_minus_x - target_x, end_minus_y - target_y, 0]) 
              
        
        if i == len(x_final) - 1:
            end_horizon_x = x_final[i - 1]
            end_horizon_y = y_final[i - 1]
        
            horizon_vector = np.array([target_x - end_horizon_x, target_y - end_horizon_y, 0])
            
        else:
            end_horizon_x = x_final[i + 1]
            end_horizon_y = y_final[i + 1]
            
            horizon_vector = np.array([end_horizon_x - target_x, end_horizon_y - target_y, 0])
              
               
        z_value = np.cross(horizon_vector, end_plus_vector)[2]
        
        if z_value <= 0:
            cv2.line(dst_img_th, (target_x, target_y), (end_plus_x, end_plus_y), (0,0,0), 5)
        else:
            cv2.line(dst_img_th, (target_x, target_y), (end_minus_x, end_minus_y), (0,0,0), 5)
        
            
    h = int(input_img.shape[0] / 5)
    w = int(input_img.shape[1] / 2)
    
    dst_img_th = dst_img_th[h : h + input_img.shape[0], w : input_img.shape[1] + w]
        
    cv2.imwrite(f"{inputdir}\\test.png", dst_img_th) #specify output folder
    
    
   
        
    
    
        
    

    
        

In [66]:
#specify input directory
inputdir="C:\\Users\\sunad\\Desktop\\exp210919_cantas\\tissue_segmentation_220305\\segmentation\\96h"
#anterior-posterior axis manually drawn
input_axis=cv2.imread(f"{inputdir}\\tissue3_axis.png", cv2.IMREAD_GRAYSCALE)
#binary image of tissue 
input_img=cv2.imread(f"{inputdir}\\tissue3_round1_c2_for_segmentation.png", cv2.IMREAD_GRAYSCALE)
    
    
        
    

    
        

In [67]:
#specify the reference points to reorder the segmented area. It should be set somewhere above the most anterior part
ref_point=[3000, 40]

#smaller the more accurate and slower (minimum 1)
skip_value=1

#The value is related with the distance between the end of line along the anterior-posterior axis. Orthogonal lines are set to this line.
slope_cal=20

#length of orthogonal slope. It should be long enough to cross over the tissue
line_length=3000


dst_img, axis = add_blank(input_img, input_axis)

pos_x_ordered, pos_y_ordered = order_points(axis)

posx_sub, posy_sub, ortho_slope_list = get_ortho_slope(pos_x_ordered, pos_y_ordered)

posx_sub_updated, posy_sub_updated, ortho_slope_updated, area_list =  get_segmented_area (posx_sub, posy_sub, ortho_slope_list)





In [70]:
#the value decides how many semgments the tissue will be divided into
dividing_num = 30

x_final, y_final, ortho_slope_final = segmenter_list(posx_sub_updated, posy_sub_updated, ortho_slope_updated, area_list)

draw_line(x_final, y_final, ortho_slope_final)





